<a href="https://colab.research.google.com/github/rodrigobernall/ds4a_group30/blob/master/scripts/EDA/origin_destination_matrix_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando las librerías

In [0]:
!pip install geopandas

In [1]:
import pandas as pd
import geopandas as gpd
import folium

# Importando los datos

In [2]:
url = 'https://www.dropbox.com/s/7ixw7pe46q3theo/green_trips_nta.csv.gz?dl=1'
green_trips_nta = pd.read_csv(url, compression='gzip')
#green_trips_nta = green_trips_nta.drop(columns=['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])

In [3]:
url = 'https://www.dropbox.com/s/k4amamqkqf7ap92/yellow_trips_nta.csv.gz?dl=1'
yellow_trips_nta = pd.read_csv(url, compression='gzip')

In [4]:
url = 'https://www.dropbox.com/s/pjovxyr8w6x6y5x/zones.csv?dl=1'
zones = pd.read_csv(url)
#zones

In [5]:
url = 'https://www.dropbox.com/s/8gys15vzjpd1ibu/demographics.csv?dl=1'
demographics = pd.read_csv(url)
#demographics

## Computing NTA centroids

In [6]:
url = 'https://www.dropbox.com/s/pt2qx83gxzpsacp/nta.json?dl=1'
nta_polygons = gpd.read_file(url)

In [7]:
url = 'https://www.dropbox.com/s/3ts67qym904tess/nta_centroids.json?dl=1'
nta_centroids = gpd.read_file(url)

## Origin-destination matrices

In [8]:
most_common_destinations_green = green_trips_nta['dropoff_nta'].value_counts()
most_common_origins_green = green_trips_nta['pickup_nta'].value_counts()

In [9]:
most_common_destinations_green

BK73    113889
MN03    104135
QN70    100472
MN34     99829
QN28     92520
         ...  
SI48        11
SI32        10
SI25         9
SI01         9
MN25         1
Name: dropoff_nta, Length: 180, dtype: int64

In [10]:
ODM_green = green_trips_nta.groupby(['pickup_nta', 'dropoff_nta']).size()
ODM_green = pd.DataFrame(ODM_green)
ODM_green = ODM_green.sort_values(by=[0], ascending=False)
ODM_green.columns = ['trips']

In [11]:
ODM_green

trips
pickup_nta dropoff_nta       
QN70       QN70         41981
BK73       BK73         40643
           BK76         31137
QN28       QN28         27368
MN11       MN03         26780
...                       ...
MN09       QN02             1
           BK27             1
MN06       SI37             1
           QN61             1
BX59       BK96             1

[14633 rows x 1 columns]

Adding the centroids of the NTAs to the ODM (Origin-Destination Matrix)

In [12]:
a = ODM_green.reset_index()
a = a.merge(nta_centroids, left_on='pickup_nta', right_on='id')
a = a.merge(nta_centroids, left_on='dropoff_nta', right_on='id', suffixes=('_pickup', '_dropoff'))
a = a.drop(columns=['id_pickup', 'id_dropoff'])
a

,pickup_nta,dropoff_nta,trips,geometry_pickup,geometry_dropoff
0,QN70,QN70,41981,POINT (40.76150 -73.91970),POINT (40.76150 -73.91970)
1,BK73,QN70,2499,POINT (40.71492 -73.95863),POINT (40.76150 -73.91970)
2,QN28,QN70,5973,POINT (40.75732 -73.88532),POINT (40.76150 -73.91970)
3,MN11,QN70,279,POINT (40.80434 -73.95129),POINT (40.76150 -73.91970)
4,BK37,QN70,303,POINT (40.67124 -73.98390),POINT (40.76150 -73.91970)
...,...,...,...,...,...
14628,QN31,SI32,1,POINT (40.73978 -73.93119),POINT (40.54034 -74.20783)
14629,BK38,SI32,6,POINT (40.69294 -73.98599),POINT (40.54034 -74.20783)
14630,BK09,SI32,1,POINT (40.69548 -73.99487),POINT (40.54034 -74.20783)
14631,BK75,SI32,1,POINT (40.69152 -73.94991),POINT (40.54034 -74.20783)


In [13]:
def create_paths_point(point_pickup, point_dropoff):
    pickup = tuple([point_pickup.x, point_pickup.y])
    dropoff = tuple([point_dropoff.x, point_dropoff.y])
    points = [pickup, dropoff]
    return points

def create_paths_dataframe(dataframe):
    long_pickup = dataframe['geometry_pickup'].apply(lambda p: p.y)
    lat_pickup = dataframe['geometry_pickup'].apply(lambda p: p.x)
    
    long_dropoff = dataframe['geometry_dropoff'].apply(lambda p: p.y)
    lat_dropoff = dataframe['geometry_dropoff'].apply(lambda p: p.x)

    point_pickup = zip(lat_pickup, long_pickup)
    point_dropoff = zip(lat_dropoff, long_dropoff)

    path = zip(point_pickup, point_dropoff)
    
    paths = []
    for i in path:
        paths.append(list(i))

    return paths

In [14]:
paths_greens = create_paths_dataframe(a.sort_values(by='trips', ascending=False))

In [15]:
a.sort_values(by='trips', ascending=False)

,pickup_nta,dropoff_nta,trips,geometry_pickup,geometry_dropoff
0,QN70,QN70,41981,POINT (40.76150 -73.91970),POINT (40.76150 -73.91970)
1293,BK73,BK73,40643,POINT (40.71492 -73.95863),POINT (40.71492 -73.95863)
1991,BK73,BK76,31137,POINT (40.71492 -73.95863),POINT (40.72951 -73.94953)
582,QN28,QN28,27368,POINT (40.75732 -73.88532),POINT (40.75732 -73.88532)
4882,MN11,MN03,26780,POINT (40.80434 -73.95129),POINT (40.81827 -73.94077)
...,...,...,...,...,...
2336,BK46,MN15,1,POINT (40.61732 -73.97033),POINT (40.76418 -73.99240)
10063,BK69,BX26,1,POINT (40.68798 -73.96236),POINT (40.83783 -73.92616)
2334,QN02,MN15,1,POINT (40.67210 -73.77304),POINT (40.76418 -73.99240)
2333,QN15,MN15,1,POINT (40.60246 -73.75532),POINT (40.76418 -73.99240)


In [16]:
folium_map = folium.Map(location=[40.738, -73.98],
                        zoom_start=13,
                        tiles="Stamen Toner")


for path in paths_greens[:500]:
    folium.PolyLine(path, color="green", weight=1.5).add_to(folium_map)

#Now we can have a scatter plot of the first 1000 data points on the above map
#for i in range(0,1000):
#    marker = folium.CircleMarker(location=[listings["latitude"][i],listings["longitude"][i]],radius=5,color="r",fill=True)
#    marker.add_to(folium_map)

folium_map